# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
# from pymongo import MongoClient
# client = MongoClient("")

from pymongo import MongoClient
dbName="companies.json"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()



### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
# Your Code
query=db.companies.find({'name':'Babelgum'})
for n in query:
    print(n['name'])

Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [5]:
# Your Code
from pymongo import ASCENDING,DESCENDING
employees=db.companies.find({'number_of_employees':{'$gt':5000}}).limit(20).sort([('number_of_employees',DESCENDING)])
for e in employees:
    print(e['name'],e['number_of_employees'])

Siemens 405000
IBM 388000
Toyota 320000
PayPal 300000
Nippon Telegraph and Telephone Corporation 227000
Samsung Electronics 221726
Accenture 205000
Tata Consultancy Services 200300
Flextronics International 200000
Safeway 186000
Sony 180500
LG 177000
Ford 171000
Boeing 160000
Digital Equipment Corporation 140000
Nokia 125000
MItsubishi Electric 107000
MItsubishi Electric 107000
Comcast 100000
Bertelsmann 100000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [6]:
# Your Code
fyears=db.companies.find({'founded_year':{'$gte':2000,'$lt':2006}},{'name':1,'founded_year':1})
for f in fyears:
    print(f['name'],f['founded_year'])
    

Wetpaint 2005
Omnidrive 2005
Digg 2004
Zoho 2005
Gizmoz 2003
StumbleUpon 2002
Helio 2005
Facebook 2004
Plaxo 2002
Technorati 2002
AddThis 2004
Veoh 2004
Jingle Networks 2005
Meetup 2002
Wesabe 2005
Jangl SMS 2005
SmugMug 2002
LifeLock 2005
Jajah 2005
Skype 2003
YouTube 2005
Pando Networks 2004
Ikan 2003
delicious 2003
Topix 2002
Jobster 2004
Revision3 2005
AllPeers 2004
AllofMP3 2000
iSkoot 2005
Steorn 2000
blinkx 2004
iContact 2003
MeeVee 2000
ClipBlast! 2005
Zlio 2005
Coghead 2003
Yelp 2004
Farecast 2003
KickApps 2004
Dailymotion 2005
RockYou 2005
Netvibes 2005
Instructables 2005
Swivel 2005
Slide 2005
Flickr 2004
Bebo 2005
TripHub 2005
FeedBurner 2004
Dogster 2004
Sway 2004
CornerWorld 2002
PBworks 2005
adBrite 2003
Loomia 2004
Meebo 2005
Newsvine 2005
iLike 2002
Jamendo 2005
Box 2005
Conduit 2005
Kaboodle 2005
LinkedIn 2003
Wink 2004
TechnologyGuide 2001
Redfin 2004
MotionDSP 2005
Techmeme 2004
Eventful 2004
Zvents 2005
oDesk 2005
Simply Hired 2004
Oodle 2004
ooma 2004
stylediary 2

Gameware Development 2003
RelevantView 2002
Android 2003
AF Telecom Sarl 2003
Amanager 2004
InvenSense 2003
Sony BMG Music Entertainment 2004
Infinera Corporation 2000
Innovolt 2005
The Icon Factory 2000
TraceWorks 2002
UNYK 2005
BioWizard 2005
Mind Candy 2003
AllMyData 2004
SilverStripe 2000
Fotolink Media 2005
Inovatis 2005
AVIS eSolutions 2000
kajeet 2003
TicketLeap 2003
Velocify 2004
FusionOps 2000
Foldera 2001
PatientsLikeMe 2004
Xerpi 2004
iNET Interactive 2002
TrustMe 2005
Hush Labs 2005
PhaseWare 2004
YourTechOnline 2000
PrimeSense 2005
Skyrider 2003
Phi Media 2005
Fantasy Sports Matrix 2005
IPEVO 2005
ChatAny 2005
Traffic Broker 2004
Control4 2003
Talend 2005
GoMobo 2005
Musopen 2005
Catzilla 2003
Segala 2002
IMAGE202 ENTERTAINMENT 2002
3Dynamics 2004
ExpertFlyer 2004
dotMobi 2005
Boardwalktech 2004
hosting365 2001
Boulevard R 2005
Andera 2000
ClairMail 2004
Waleg 2003
BioIQ 2005
Swamp Fox 2000
TouchLocal 2004
Facilitas 2005
GuardID 2004
IP Commerce 2005
Dilithium Networks 200

Idiro 2004
Blueface 2004
CreativeHost Hospedagem 2002
ConSentry Networks 2003
LogLogic 2002
Sipera Systems 2003
SEEDCORE GROUP 2005
Aruba Networks 2002
The Solution Group 2002
3biz 2005
Zonablu Networks 2005
3Sigma 2000
Zen Web Solutions 2000
TNC 2004
Web CEO 2000
Semantra 2003
nextep 2000
PageTraffic 2002
Quantum Voyage 2004
shanghai junda instrument co,ltd 2003
PROJECT DIRT 2005
Qpyn 2003
Brick Marketing 2005
Ziva Software 2005
Nabbit 2005
OnVantage 2004
Meritia mobile 2004
Endavo Media and Communications 2004
Amethon Solutions 2000
Vidiac 2004
Jobmixi 2004
Viocorp 2002
Trizeon 2004
Alwayswater 2002
Atalasoft 2002
Intellectual Ventures 2000
SEO Services Group 2000
Open Lab 2001
eSchedule 2005
Securent 2004
Jabber 2000
InfoWeapons 2002
Optiant 2000
Cupid XL 2005
Frontline Selling 2001
Elemica 2000
SK Net Service Company Ltd 2001
Vertica Systems 2005
Alpha SEP 2000
Exaprotect 2004
Ratchet 2004
Infinity Pharmaceuticals 2001
Voice123 2003
Club Scene Network 2004
Facebookster 2003
CityVot

Team Rubber 2001
SpaceClaim 2005
OpVista 2000
MiNOWireless 2004
Sophic 2004
Assembla 2005
Mazu Networks 2000
Traiana 2000
Luminary Micro 2004
Staccato Communications 2002
Eyedea Lab 2004
Bridgestream 2000
Carrier IQ 2005
Marble Security 2005
Gennio 2005
ZVUE 2003
Acopia Networks 2002
Hidden City Games 2004
Tobii Technology 2001
AdsTamil 2005
VidSys 2005
Xelerated 2000
Neoconix 2003
Visage Mobile 2001
Travel Distribution Systems 2002
TechForward 2005
Squeejee 2000
i-Jet Media 2005
Rigel Networks 2004
OnRequest Images 2002
Vista Research 2001
Celeno 2005
NextHop Technologies 2000
SUPERAntiSpyware 2002
Code Authority Custom Software 2001
Litepoint 2001
VirtuOz 2002
Jimojo 2005
Neverblue 2004
CipherMax 2000
Qihoo 360 Technology 2005
Medio 2004
LuckyDogJobs 2004
CipherTrust 2000
Septagon Studios 2005
Sittercity 2001
TheMarkets 2000
Etology 2005
Avnera 2004
Manifest Digital 2001
Technology In Ministry 2005
Bharosa 2003
Invarium 2003
Oversi 2004
BigSoccer 2005
Phu Concepts 2005
MerchantRun 20

Brainsgate 2000
Bigpoint 2002
PINC Solutions 2004
PrecisionPoint Software 2002
ICU Global 2002
PayPay 2003
Santhos 2005
ZoomSystems 2002
Puppet Labs 2005
Catalyst Biosciences 2003
Accera 2001
Submittal Exchange 2003
Zugara 2001
Costumzee 2005
Ampere Software Private Limited 2005
City Network Hosting 2002
EGEN 2002
Intermolecular 2004
Carpathia Hosting 2003
Pontis 2004
ocProducts 2004
Enpirion 2001
Joomla 2005
Exalt Communications 2004
Visiogen 2001
Audience 2000
Palyon Medical 2004
CallFire 2005
Marinus Pharmaceuticals 2003
iNetGiant 2002
Our Memories in Motion 2004
Kyero 2003
Intradigm Corporation 2000
Global Data Solutions 2000
Green Wave Holidays 2003
OpTier 2005
dream ad 2004
Savant 2002
Hythiam 2000
NetXen 2002
New Energy Finance 2004
Grubby Games 2004
Prime Time Communications 2005
Azingo 2005
Premier Power 2001
mediaburst 2000
Oesia 2000
Altivation Software 2001
Video Game 911 2005
Axiom Microdevices 2002
All India Guide 2003
Varonis Systems 2005
Zymeworks 2004
Schoolwires 2000


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [7]:
# Your Code
valuation=db.companies.find({"$and":[{"ipo.valuation_amount":{"$gte":100000000}},{"founded_year":{"$lt":2010}}]},{"name":1,"ipo":1})
for v in valuation:
    print(v["name"],v["ipo"])

Twitter {'valuation_amount': 18100000000, 'valuation_currency_code': 'USD', 'pub_year': 2013, 'pub_month': 11, 'pub_day': 7, 'stock_symbol': 'NYSE:TWTR'}
Facebook {'valuation_amount': 104000000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 5, 'pub_day': 18, 'stock_symbol': 'NASDAQ:FB'}
Yelp {'valuation_amount': 1300000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 3, 'pub_day': 2, 'stock_symbol': 'NYSE:YELP'}
LinkedIn {'valuation_amount': 9310000000, 'valuation_currency_code': 'USD', 'pub_year': 2011, 'pub_month': 7, 'pub_day': 20, 'stock_symbol': 'NYSE:LNKD'}
Brightcove {'valuation_amount': 290000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 2, 'pub_day': 17, 'stock_symbol': 'NASDAQ:BCOV'}
Amazon {'valuation_amount': 100000000000, 'valuation_currency_code': 'USD', 'pub_year': 1997, 'pub_month': 5, 'pub_day': None, 'stock_symbol': 'NASDAQ:AMZN'}
KIT digital {'valuation_amount': 235000000, 'valuation_currency_code': '

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [ ]:
# Your Code
employeesnumber=db.companies.find({"$and":[{"number_of_employees":{"$lt":1000}},{"founded_year":{"$lt":2005}}]}).limit(10).sort([("number_of_employees", DESCENDING)])
for e in employeesnumber:
    print(e["name"],e["number_of_employees"],e["founded_year"])

### 6. All the companies that don't include the `partners` field.

In [ ]:
# Your Code
partnersnotexist=db.companies.find({"partners":{"$exists":"false"}})
for x in partnersnotexist:
    print(x["name"])

### 7. All the companies that have a null type of value on the `category_code` field.

In [ ]:
# Your Code
category_ull=db.companies.find({"category_code":None}) # tambien se puede poner {"type":"null"}}
for x in categorynull:
    print(x["name"])

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [ ]:
# Your Code
employees_number=db.companies.find({"number_of_employees":{"$gte": 100,"$lt":1000}},{"name":1,"number_of_employees":1})
for e in employees_number:
    print(e["name"],e["number_of_employees"])

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
# Your Code
ipo_price=db.companies.find({"ipo.valuation_amount":{"exists":"true"}}).sort([("ipo.valuation_amount", DESCENDING)])
for n in ipo_price:
    print(n["name"],n["ipo"]["valuation_amount"])

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
# Your Code
top_companies=db.companies.find({"number_of_employees":{"exists":True}}).limit(10).sort([("number_of_employees", DESCENDING)])
for e in top_companies:
    print(e["name"],e["number_of_employees"])

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
# Your Code
second_semester=db.companies.find({"founded_month":{"$gte": 6,"$lt":12}}).limit(1000)
for e in second_semester:
    print(e["name"],e["founded_month"])

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
# Your Code
companies=db.companies.find({"$and":[{"founded_year":{"$lt": 2000}},{"acquisition.price_amount":{"$gt": 1000000}}]})
for e in companies:
    print(e["name"],e["founded_year"],e["acquisition"]["price_amount"])

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code
companies2=db.companies.find({"acquisition.acquired_year":{"$gt": 2010}},{"acquisition.price_amount":1,"name":1}).sort([("acquisition.price_amount", DESCENDING)])
for e in companies2:
    print(e["name"],e["acquisition"])

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code
companies_year=db.companies.find({"founded_year":{"exists":True}},{"name":1,"founded_year":1}).sort([("founded_year", DESCENDING)]) 
for x in companies_year:
    print(x["name"],x["founded_year"])

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code
cursor=db.companies.find({"founded_day":{"$lte":7}}).limit(10).sort([("acquisition.price_amount", DESCENDING)]) 
for x in cursor:
    print(x["name"],x["founded_day"],x["acquisition"]["price_amount"])

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code